In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [2]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)

[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


In [3]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)

In [4]:
%timeit jnp.dot(x, x.T).block_until_ready()

8.54 ms ± 441 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
import numpy as np
x = np.random.normal(size=(size, size)).astype(np.float32)

In [6]:
%timeit jnp.dot(x, x.T).block_until_ready()

26.1 ms ± 99.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
x = device_put(x)
%timeit jnp.dot(x, x.T).block_until_ready()

8.87 ms ± 48.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
def selu(x, alpha=1.67, lmbda=1.05):
    return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

In [9]:
x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

498 µs ± 67.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

109 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
import jax

In [13]:
jax.local_devices()

[gpu(id=0), gpu(id=1)]

In [15]:
def sum_logistic(x):
    return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

In [16]:
x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

[0.25       0.19661194 0.10499357]


In [17]:
def first_finite_differences(f, x):
    eps = 1e-3
    return jnp.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps) for v in jnp.eye(len(x))])

In [18]:
print(first_finite_differences(sum_logistic, x_small))

[0.24998187 0.1965761  0.10502338]


In [19]:
print(grad(jit(grad(jit(grad(sum_logistic)))))(1.0))

-0.0353256


In [20]:
from jax import jacfwd, jacrev

In [21]:
def hessian(fun):
    return jit(jacfwd(jacrev(fun)))

In [22]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
    return jnp.dot(mat, v)

In [23]:
def naively_batched_apply_matrix(v_batched):
    return jnp.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
2.16 ms ± 371 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
@jit
def batched_apply_matrix(v_batched):
    return jnp.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched
23.7 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [25]:
@jit
def vmap_batched_apply_matrix(v_batched):
    return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Auto-vectorized with vmap
23.5 µs ± 111 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
